# Install TensorFlow.
*   Install TensorFlow-1.14.0 with CUDA-10 support.
*   Install Keras-2.2.4, which compatible with TensorFlow-1.14.0.

In [0]:
%tensorflow_version 1.x
!pip uninstall tensorflow -y
!pip uninstall tensorflow-gpu -y
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install --upgrade keras==2.2.4

# Use installed TensorFlow version by running Runtime -> Restart runtime.

# Install keras-contrib module.

In [0]:
%tensorflow_version 1.x
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

# Import python modules.

In [0]:
%tensorflow_version 1.x
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D

In [0]:
from keras.layers import LeakyReLU
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

# Create the weight initializer.
* Gaussian distribution - N(0.0, 0.02).

In [0]:
from keras.initializers import RandomNormal

initializer = RandomNormal(mean=0.0, stddev=0.02)

# Create the optimizer.
*   Adam optimizer.
*   Learning rate = 0.0002 - First 100 epochs.
*   Learning rate is decreased from 0.0002 to 0.0 in 100 epochs.

In [0]:
from keras.optimizers import Adam

optimizer = Adam(lr=0.0002, beta_1=0.5)

# Create the discriminator model.

In [0]:
def create_discriminator(image_shape):
	# Source image input.
	input_image = Input(shape=image_shape)
 
	# Create layer C64.
	layer = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer)(input_image)
	layer = LeakyReLU(alpha=0.2)(layer)
 
	# Create layer C128.
	layer = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer)(layer)
	layer = InstanceNormalization(axis=-1)(layer)
	layer = LeakyReLU(alpha=0.2)(layer)
 
	# Create layer C256.
	layer = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer)(layer)
	layer = InstanceNormalization(axis=-1)(layer)
	layer = LeakyReLU(alpha=0.2)(layer)
 
	# Create layer C512.
	layer = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer)(layer)
	layer = InstanceNormalization(axis=-1)(layer)
	layer = LeakyReLU(alpha=0.2)(layer)
 
	# Create second last layer.
	layer = Conv2D(512, (4,4), padding='same', kernel_initializer=initializer)(layer)
	layer = InstanceNormalization(axis=-1)(layer)
	layer = LeakyReLU(alpha=0.2)(layer)
 
	# Create the discriminator output layer.
	model_output = Conv2D(1, (4,4), strides=(1,1), padding='same', kernel_initializer=initializer)(layer)
 
	# Create the discriminator model.
	discriminator_model = Model(input_image, model_output)
 
	# Compile the discriminator model.
	# Loss - MSE - Weighted by 1/2 = 0.5
	discriminator_model.compile(loss='mse', optimizer=optimizer, loss_weights=[0.5])
	return( discriminator_model )

In [0]:
# Define input image shape.
image_shape = (256, 256, 3)
# Create the discriminator model.
model = create_discriminator(image_shape)
# Show the model summary.
model.summary()

# Define the ResNet block.

In [0]:
from keras.layers import Activation
from keras.layers import Concatenate

In [0]:
def create_resnet_block(number_of_filters, input_layer):

	# Create first convolutional layer.
	block_output = Conv2D(number_of_filters, (3,3), padding='same', kernel_initializer=initializer)(input_layer)
	block_output = InstanceNormalization(axis=-1)(block_output)
	block_output = Activation('relu')(block_output)
 
	# Create second convolutional layer.
	block_output = Conv2D(number_of_filters, (3,3), padding='same', kernel_initializer=initializer)(block_output)
	block_output = InstanceNormalization(axis=-1)(block_output)
 
	# Concatenate and merge channel-wise with input layer.
	block_output = Concatenate()([block_output, input_layer])
	return(block_output)

# Create the generator model.

In [0]:
from keras.layers import Conv2DTranspose

In [0]:
def create_generator(image_shape=(256,256,3), resnet_blocks=9):

	# Create input image.
	input_image = Input(shape=image_shape)
 
  ######################################################################
  # c7s1-k - Conv2D-InstanceNormalization-ReLU block. 
	#        - Conv2D 7×7 with k filters and stride 1.
  
	# dk     - Conv2D-InstanceNormalization-ReLU block.
	#        - Conv2D 3×3 with k filters and stride 2.

	# Rk     - Residual block 
	#        - 2 3×3 Conv2D layers with k filters on both layers.
	
	# uk     - Fractional-strided Conv2D-InstanceNormalization-ReLU block. 
	#        - Conv2D 3×3 with k filters and stride 1/2.
  ######################################################################

	# Create c7s1-64 block.
	layer = Conv2D(64, (7,7), strides=(1,1), padding='same', kernel_initializer=initializer)(input_image)
	layer = InstanceNormalization(axis=-1)(layer)
	layer = Activation('relu')(layer)
 
	# Create d128 block.
	layer = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=initializer)(layer)
	layer = InstanceNormalization(axis=-1)(layer)
	layer = Activation('relu')(layer)
 
	# Create d256 block.
	layer = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=initializer)(layer)
	layer = InstanceNormalization(axis=-1)(layer)
	layer = Activation('relu')(layer)
 
	# Create R256 blocks.
	for _ in range(resnet_blocks):
		layer = create_resnet_block(256, layer)
	
	# Create u128 block.
	layer = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=initializer)(layer)
	layer = InstanceNormalization(axis=-1)(layer)
	layer = Activation('relu')(layer)
 
	# Create u64 block.
	layer = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=initializer)(layer)
	layer = InstanceNormalization(axis=-1)(layer)
	layer = Activation('relu')(layer)
 
	# Create c7s1-3 block.
	layer = Conv2D(3, (7,7), strides=(1,1), padding='same', kernel_initializer=initializer)(layer)
	layer = InstanceNormalization(axis=-1)(layer)
	output_image = Activation('tanh')(layer)
 
	# Create the generator model.
	generator_model = Model(input_image, output_image)
	return( generator_model)

In [0]:
# Define input image shape.
image_shape = (256, 256, 3)
# Create the generator model.
model = create_generator(image_shape)
# Show the model summary.
model.summary()

# Create a composite model for updating the generators using adversarial and cycle loss.

In [0]:
def create_composite_model(generator_A_to_B, discriminator_2, generator_B_to_A, image_shape):
	# Generator for domain A to B is trainable.	
	generator_A_to_B.trainable = True

	# Discriminator for domain B is NOT trainable.
	discriminator_2.trainable = False

	# Generator for domain B to A is NOT trainable.
	generator_B_to_A.trainable = False

	# Define domain B discriminator output.
	generator_input = Input(shape=image_shape)
	generator_A2B_output = generator_A_to_B(generator_input)
	discriminator_B_output = discriminator_2(generator_A2B_output)
 
	# Define identity output.
	identity_input = Input(shape=image_shape)
	identity_output = generator_A_to_B(identity_input)
 
	# Define forward cycle output.
	forward_cycle_output = generator_B_to_A(generator_A2B_output)
 
	# Define backward cycle output.
	generator_B2A_output = generator_B_to_A(identity_input)
	backward_cycle_output = generator_A_to_B(generator_B2A_output)
 
	# Create the composite model.
	composite_model = Model([generator_input, identity_input], 
	                        [discriminator_B_output, identity_output, 
													 forward_cycle_output, backward_cycle_output])
 
	# Compile the composite model with weighted loss using least squares loss and L1 loss.
	composite_model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=optimizer)
	return(composite_model)

In [0]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# choose random instances
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = np.ones((n_samples, patch_shape, patch_shape, 1))
	return X, y

In [0]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
	# generate fake instance
	X = g_model.predict(dataset)
	# create 'fake' class labels (0)
	y = np.zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [0]:
# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif np.random.random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = np.random.randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return np.asarray(selected)

In [0]:
# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset, number_of_epochs=100, batch_size=1):

	# determine the output square shape of the discriminator
	n_patch = d_model_A.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# prepare image pool for fakes
	poolA, poolB = list(), list()
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / batch_size)
	# calculate the number of training iterations
	n_steps = bat_per_epo * number_of_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		X_realA, y_realA = generate_real_samples(trainA, batch_size, n_patch)
		X_realB, y_realB = generate_real_samples(trainB, batch_size, n_patch)
		# generate a batch of fake samples
		X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
		# update fakes from pool
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
		# update generator B->A via adversarial and cycle loss
		g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
		# update discriminator for A -> [real/fake]
		dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
		# update generator A->B via adversarial and cycle loss
		g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
		# update discriminator for B -> [real/fake]
		dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
		# summarize performance
		print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))    

In [0]:
import os
import keras

dataset_url = 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip'

zip_file = keras.utils.get_file(origin=dataset_url,
                                   fname="horse2zebra.zip",
                                   extract=True)

#"ae_photos" "apple2orange" "summer2winter_yosemite" "horse2zebra" "monet2photo" "cezanne2photo" "ukiyoe2photo" "vangogh2photo" "maps" "cityscapes" "facades" "iphone2dslr_flower" "ae_photos" 

base_dir = os.path.join(os.path.dirname(zip_file), 'horse2zebra')

In [0]:
!ls -al /root/.keras/datasets/horse2zebra

In [0]:
import cv2
from glob import glob
import numpy as np

def load_data(domain, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('%s/%s/*' % (base_dir,data_type))        

        imgs = []
        for img_path in path:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR).astype(np.float)
            if not is_testing:
                img = cv2.resize(img, (256, 256))

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = cv2.resize(img, (256, 256))
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

In [0]:
train_horses = load_data('A', is_testing=False)
train_zebras = load_data('B', is_testing=False)

print(len(train_horses))
print(len(train_zebras))

In [0]:
test_horses = load_data('A', is_testing=True)
test_zebras = load_data('B', is_testing=True)    

print(len(test_horses))
print(len(test_zebras))

In [0]:
# load a dataset as a list of two numpy arrays
dataset = [train_horses, train_zebras]

# input shape
image_shape = (256,256,3)
# generator: A -> B
g_model_AtoB = create_generator(image_shape)
# generator: B -> A
g_model_BtoA = create_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = create_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = create_discriminator(image_shape)

# composite: A -> B -> [real/fake, A]
c_model_AtoB = create_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = create_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

number_of_epochs = 1
batch_size = 1
# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset, number_of_epochs, batch_size)

In [0]:
g_model_AtoB.save_weights('g_model_AtoB.h5')
g_model_BtoA.save_weights('g_model_BtoA.h5')

In [0]:
output_images = g_model_AtoB.predict(test_horses)

for input_image, output_image in zip(test_horses, output_images):
  input_image = (input_image + 1) * 127.5
  output_image = (output_image + 1) * 127.5
  cv2.imwrite('input_image.png', input_image)
  cv2.imwrite('output_image.png', output_image)